In [1]:
import fiftyone as fo
import numpy as np

dataset = fo.zoo.load_zoo_dataset(
              "open-images-v7",
              split="train",
              label_types=["detections"],
              classes=["Cat"],
            #   max_samples=10,
          )

Necessary images already downloaded
Existing download of split 'train' is sufficient
Loading existing dataset 'open-images-v7-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
import cv2

In [ ]:
# batching is not necessary if we're using fasterrcnn_resnet50_fpn.

# batch_size = 100
# n_samples = len(dataset)


# x_size = 267
# y_size = 326
# # (267 x 326) is the number of pixels in the smallest images
# # TODO: we downsize all images? i guess
# batched_data = np.ndarray((n_samples // batch_size, batch_size, y_size, x_size, 3))
# batched_boxes = np.ndarray((n_samples // batch_size, batch_size, 4))

# for batch_index in range(n_samples // batch_size):
#     for i, sample in enumerate(dataset[batch_index * batch_size:(batch_index + 1) * batch_size]):
#         batched_data[batch_index, i] = cv2.resize(cv2.imread(sample['filepath']), (x_size, y_size))

#         for detection in sample['ground_truth']['detections']:
#             if detection['label'] == 'Cat':
#                 batched_boxes[batch_index, i] = detection['bounding_box']
#                 break


In [ ]:
unbatched_data = np.ndarray((n_samples, y_size, x_size, 3))
unbatched_boxes = np.ndarray((n_samples, 4))
for i, sample in enumerate(dataset):
    unbatched_data[i] = cv2.resize(cv2.imread(sample['filepath']), (x_size, y_size))
    for detection in sample['ground_truth']['detections']:
            if detection['label'] == 'Cat':
                unbatched_boxes[i] = detection['bounding_box']
                break

targets = []
for i in range(n_samples):
     d = {}
     d['boxes'] = torch.tensor(np.array(unbatched_boxes[i]))
     d['labels'] = ['Cat']
     targets.append(d)

TypeError: can't assign a numpy.ndarray to a torch.DoubleTensor

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
print(unbatched_data.shape, unbatched_boxes.shape)
output = model(unbatched_data, targets)
model.eval()